In [1]:
import numpy
import keras
from keras.models import Model
from keras.layers import Activation
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
import os

Using TensorFlow backend.


In [2]:
root_path = r"C:\Users\vikky\Documents\Third Year\Perk Lab\181101_Spine\181101_Spine"

train_path = os.path.join(root_path, "train")
valid_path = os.path.join(root_path, "valid")
test_path = os.path.join(root_path, "test")

In [3]:
image_size = 32
train_batch_size = 100

In [4]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True,
    batch_size=train_batch_size)

valid_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True,
    batch_size=train_batch_size)

test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=(image_size, image_size),
    color_mode='grayscale',
    classes=["pos", "neg"],
    shuffle=True)

Found 2037 images belonging to 2 classes.
Found 715 images belonging to 2 classes.
Found 747 images belonging to 2 classes.


In [5]:
#Create train, validation, and test batches
train_batches.batch_size = train_batches.n
x_train, y_train = next(train_batches)

valid_batches.batch_size = valid_batches.n
x_valid, y_valid = next(valid_batches)

test_batches.batch_size = test_batches.n
x_test, y_test = next(test_batches)

In [6]:
print("x_test.shape: {}".format(x_test.shape))
print("y_test.shape: {}".format(y_test.shape))

print("x_valid.shape: {}".format(x_valid.shape))
print("y_valid.shape: {}".format(y_valid.shape))

print("x_train.shape: {}".format(x_train.shape))
print("y_train.shape: {}".format(y_train.shape))

x_test.shape: (747, 32, 32, 1)
y_test.shape: (747, 2)
x_valid.shape: (715, 32, 32, 1)
y_valid.shape: (715, 2)
x_train.shape: (2037, 32, 32, 1)
y_train.shape: (2037, 2)


In [7]:
input_img = Input(shape = (32, 32, 1))

In [8]:
#Build the layers
def inception(input_img):
    tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
    tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
    tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
    tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
    tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

    output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)
    return output

In [9]:
#Create the model
def inceptionModel():

    inputs = Input(shape = (32, 32, 1))

    x = inception(inputs)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    predictions = Dense(2, activation='softmax')(x)

    model = Model(input=inputs, output=predictions)

    model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=Adam(lr=0.001),
                 metrics=['accuracy'])
    return model

In [10]:
modelA = inceptionModel()
modelA.fit(x_train, y_train, epochs=10, shuffle=True,  validation_split=0.0, validation_data=(x_valid, y_valid))
score = modelA.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

C:\Users\vikky\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # This is added back by InteractiveShellApp.init_path()


Train on 2037 samples, validate on 715 samples
Epoch 1/10
2037/2037 [==============================] - 116s 57ms/step - loss: 0.4055 - acc: 0.8336 - val_loss: 0.3924 - val_acc: 0.8378
Epoch 2/10
2037/2037 [==============================] - 112s 55ms/step - loss: 0.1280 - acc: 0.9539 - val_loss: 0.4136 - val_acc: 0.8462
Epoch 3/10
2037/2037 [==============================] - 115s 56ms/step - loss: 0.0763 - acc: 0.9701 - val_loss: 0.4409 - val_acc: 0.8448
Epoch 4/10
2037/2037 [==============================] - 126s 62ms/step - loss: 0.0693 - acc: 0.9710 - val_loss: 0.5973 - val_acc: 0.8000
Epoch 5/10
2037/2037 [==============================] - 114s 56ms/step - loss: 0.0293 - acc: 0.9902 - val_loss: 0.5560 - val_acc: 0.8266
Epoch 6/10
2037/2037 [==============================] - 115s 56ms/step - loss: 0.0360 - acc: 0.9828 - val_loss: 0.7199 - val_acc: 0.8056
Epoch 7/10
2037/2037 [==============================] - 121s 60ms/step - loss: 0.0319 - acc: 0.9897 - val_loss: 0.6467 - val_acc: 0